# My GMV ETF Portfolio

In [17]:
from zipline import run_algorithm
from zipline.api import order_target_percent, symbol, schedule_function, date_rules, time_rules

from datetime import datetime
import pytz 

import matplotlib.pyplot as plt 

import pyfolio as pf


In [18]:
def initialize(context): 
    context.universe = [
    {   'isin': 'IE00BKM4GZ66', 
        'symbol': 'IS3N.F',
        'weight': 0.04,
        'name': 'iShares Core MSCI Emerging Markets IMI UCITS'},

    {   'isin': 'IE00BP3QZB59', 
        'symbol': 'IS3S.DE',
        'weight': 0.055,
        'name': 'iShares MSCI World Value Factor UCITS'},

    {   'isin': 'IE00BF4RFH31', 
        'symbol': 'IUSN.F',
        'weight': 0.08,
        'name': 'iShares MSCI World Small Cap UCITS USD Acc'},

    {   'isin': 'IE00BL25JP72', 
        'symbol': 'XDEM.DE',
        'weight': 0.21,
        'name': 'db x-trackers MSCI World Mom Factor DR 1C'},

    {   'isin': 'IE00BL25JL35', 
        'symbol': 'XDEQ.DE',
        'weight': 0.17,
        'name': 'db x-trackers MSCI World Quality Factor DR 1C', 
        'class': 'etf' }
    ]
        # Schedule the daily trading routine for once per month
    schedule_function(handle_data, date_rules.month_start(0), time_rules.market_close())

In [19]:
def handle_data(context, data):   

    for asset in context.universe:
        # For all positions in the universe allocate the weight
        #
        position = symbol(asset['symbol'])
        weight = asset['weight']

        if data.can_trade(position):
            order_target_percent(position, weight)


In [20]:
def analyze(context, perf):
    returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(perf)
    pf.create_returns_tear_sheet(returns, benchmark_rets=None)

In [21]:
# Run the backtest
#
start = datetime(2018, 1, 1, tzinfo=pytz.UTC)
end = datetime(2020, 7, 21, tzinfo=pytz.UTC)

result = run_algorithm(
    start=start,
    end=end,
    initialize=initialize,
    analyze=analyze,
    handle_data=handle_data,
    capital_base=10000,
    data_frequency='daily',
    bundle='database_bundle'
)

SymbolNotFound: Symbol 'IE00BKM4GZ66' was not found.